In [1]:
import numpy as np
from scipy.sparse import csr_matrix

from qiskit_dynamics.signals import SignalList,Signal
from qiskit_dynamics.models import GeneratorModel,LindbladModel

In [2]:
r = lambda *args: np.random.uniform(-1,1,args)+1j*np.random.uniform(-1,1,args)
cf = lambda a,b: print(np.allclose(a,b))
def disp(to_display,decimals=2):
    try:
        mat = to_display.copy()
        for s in mat.shape:
            mat = mat[:min([3,s])]
            mat = mat.transpose([j for j in range(1,len(mat.shape))]+[0])
        print(np.round(mat,decimals))
    except: 
        print(to_display)

We will now initialize all of our arrays for testing. We will work with a Hilbert space dimension of $n$, with $k$ terms in our Hamiltonian expression, $m$ dissipator terms.

In [3]:
n = 64 # Hilbert space dimension. Small to enable vectorized operators to fit in memory.
k = 8 # number of Hamiltonian components
m = 4 # number of dissipator terms
num_states = 16 # for testing vectorization with multiple states, how many states are there?

frame_op = r(n,n)
frame_op = frame_op - frame_op.conj().transpose()
frame_diag = 1j*r(n).imag

t = r().real

ham_terms = r(k,n,n)
drift = r(n,n)
ham_sigs = SignalList([Signal(r(),r(),r()) for i in range(k)])
ham_sig_vals = ham_sigs(t)

ham = np.tensordot(ham_sig_vals,ham_terms,axes=1)+drift

ham_terms = ham_terms + ham_terms.transpose([0,2,1])
dis_terms = r(m,n,n)
dis_sigs = SignalList([Signal(r(),r(),r()) for i in range(m)])
dis_sig_vals = dis_sigs(t)

state_vec = r(n)
state_vecs = r(n,num_states)
dens_mat = r(n,n)
dens_matrices = r(num_states,n,n)

The first question I'll attempt to answer is the following: When we conjugate an operator in the rotating frame as $e^{-Ft}H_je^{Ft}$, we typically use the fact that $F$ is diagonal to write this not as a matrix conjugation, but as elementwise multiplication by some $\Delta:e^{-Ft}H_je^{Ft} = \Delta\circ H_j, \Delta_{ij}=e^{(-d_i+d_j)t}$. But just how much faster is this process?

In [4]:
pexp = np.exp(frame_diag)
nexp = np.exp(-frame_diag)

cf(np.diag(nexp) @ ham_terms @ np.diag(pexp),np.outer(nexp,pexp) * ham_terms)
cf(np.diag(nexp) @ ham_terms @ np.diag(pexp),(nexp.reshape(n,1)*pexp)*ham_terms)
%timeit np.diag(nexp) @ ham_terms @ np.diag(pexp)
%timeit np.outer(nexp,pexp) * ham_terms
%timeit (nexp.reshape(n,1)*pexp)*ham_terms

True
True
248 µs ± 29.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
79.2 µs ± 4.78 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
65.9 µs ± 6.36 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


What about when we wish to evaluate not the Hamiltonian directly, but the Hamiltonian applied to a state $v$? In this case, it might be faster to compute $e^{Ft}v$, then $H(e^{Ft}v)$, then $e^{-Ft}(H(e^{Ft}v))$. Because $e^{\pm Ft}$ are diagonal operators, it is simple to apply them to a vector, which can be accomplished by elementwise multiplication. 

In [5]:
%timeit (np.outer(nexp,pexp) * ham) @ state_vec
%timeit pre_rot = np.diag(pexp) @ state_vec; apply_ham = ham @ pre_rot; post_rot = np.diag(nexp) @ apply_ham
%timeit nexp * (ham @ (pexp * state_vec))

20.5 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
16.9 µs ± 212 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.5 µs ± 96.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


What about when we store multiple state-vectors as a matrix? In this case, we have to do some odd array broadcasting to make things come out correctly, and the results will depend on the number of states at which we wish to simultaneously evaluate everything. With that said, it does seem that it is generally still faster to do elementwise multiplication, even up to the point that there are as many states as the dimension of the Hilbert space, which is a reasonable upper bound for the number of states a user might want to simultaneously evaluate. 

In [6]:
cf((np.outer(nexp,pexp) * ham) @ state_vecs,nexp.reshape(n,1) * (ham @ (pexp.reshape(n,1)*state_vecs)))
%timeit (np.outer(nexp,pexp) * ham) @ state_vecs
%timeit nexp.reshape(n,1) * (ham @ (pexp.reshape(n,1)*state_vecs))

%timeit (np.outer(nexp,pexp) * ham) @ dens_mat # for if we had ~n states evaluating simultaneously
%timeit nexp.reshape(n,1) * (ham @ (pexp.reshape(n,1)*dens_mat))

True
31.4 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
16.5 µs ± 339 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
42.4 µs ± 3.22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
39.4 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


One common operation that is performed somewhat regularly involves subtracting a (diagonal) frame object from some matrix. Because we are modifying only the diagonal, it may be a bit wasteful to subtract two full arrays (which should scale with $n^2$) rather than directly manipulating the diagonal (scaling with $n$). Let's contrast these two: 

In [7]:
%timeit ham - np.diag(frame_diag)
%timeit np.fill_diagonal(ham,ham.diagonal()-frame_diag)

8.25 µs ± 176 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.26 µs ± 68.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Although the second implementation can be much faster, we have chosen to stick with the first implementation because it is a bit more readable, and because the subtracting of a diagonal entry tends to occur only when a frame is set, not during the RHS calculation that needs to be extremely fast. 

Something that may end impacting performance is the addition of a drift term. In particular, when we calculate e.g. the Hamiltonian of a system, we have to compute $\sum_j s_j H_j + H_d$. Because of the way that `np.tensordot` works, it is conceivable that it is faster to simply append a 1 to the list of signal values the calculation object receives, and then compute the Hamiltonian using an augmented list of Hamiltonian terms, of which $H_d $ is part. This may be able to push more of the calculations onto `tensordot`, which in turn could lead to faster speeds, depending on how optimized it is. 

In [8]:
tmp = np.append(ham_terms,[drift],axis=0) # can be done outside of the RHS evaluation step

%timeit np.tensordot(ham_sig_vals,ham_terms,axes=1)+drift
%timeit np.tensordot(np.append(ham_sig_vals,[1]),tmp,axes=1)

39.6 µs ± 711 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
44.4 µs ± 3.03 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


I should note that this advantage tends to evaporate as you go to either large Hilbert space dimension or many Hamiltonian terms. E.g. with $n=2^{11}$ and $k=50$, the first implementation is actually slower by about 3\%. Because it seems to be a wash for larger spaces, we have chosen to avoid complication in the code, and use the first implementation. 

# Vectorized Lindblad Evaluation

We will now have a look at the relative speeds of evaluation for Lindblad evaluation, where we use dense matrices in both cases (since our terms are not mathematically sparse, we expect sparse matrices to perform worse).

In [9]:
# non-vectorized Lindblad
lind_model = LindbladModel(ham_terms,ham_sigs,0*dis_terms,dis_sigs,0*drift,frame=None,evaluation_mode="dense")
%timeit lind_model(t,dens_mat)

# vectorized Lindblad
vec_lind_model = LindbladModel(ham_terms,ham_sigs,0*dis_terms,dis_sigs,0*drift,frame=None,evaluation_mode="dense_vectorized")
vectorized_dens_mat = dens_mat.flatten(order="F")
%timeit vec_lind_model(t,vectorized_dens_mat)
cf(vec_lind_model(t,vectorized_dens_mat),lind_model(t,dens_mat).flatten(order="F"))

1.69 ms ± 97.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
431 ms ± 8.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
True


We can of course add a rotating frame to this, but it won't make a huge difference to evaluation time–at least at this Hilbert space dimension. This is primarily because the process of bringing a state into/out of the rotating frame is done quite efficiently when provided a state. 

In [10]:
lind_model.frame = frame_op
vec_lind_model.frame = frame_op

In [11]:
lm1 = lind_model(t,dens_mat)
lm2 = vec_lind_model(t,vectorized_dens_mat)
cf(lm1.flatten(order="F"),lm2)

%timeit lind_model(t,dens_mat)
%timeit vec_lind_model(t,vectorized_dens_mat)

%timeit lind_model.frame.operator_into_frame(t,dens_mat,operator_in_frame_basis=False,return_in_frame_basis=False,vectorized_operators=False)
%timeit vec_lind_model.frame.operator_into_frame(t,vectorized_dens_mat,operator_in_frame_basis=False,return_in_frame_basis=False,vectorized_operators=True)

True
2.01 ms ± 75 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
425 ms ± 6.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
204 µs ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
236 µs ± 8.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Because the vectorized Lindblad model can be evaluated without a state, it may be tempting to calculate the generator, then apply it to the state. This process, however, has to use _much_ slower algorithms, because it can't do e.g. pre- and post- rotation. This makes it _much_ slower than standard vectorized evaluation. 

In [12]:
vec_lind_model.frame = frame_op
val1=vec_lind_model(t) @ vectorized_dens_mat
val2=vec_lind_model(t,vectorized_dens_mat)
cf(val1,val2)
%timeit vec_lind_model(t) @ vectorized_dens_mat
%timeit vec_lind_model(t,vectorized_dens_mat)

True
4.1 s ± 62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
427 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Sparse Evaluation

Thus far, we have been using completely random Hamiltonian components $H_j$. Commonly, however, these components are _not_ completely random, but have a fair bit of structure. A particularly common case involves these components being mathematically sparse. In this case, it may be quite useful for the user to compute everything using sparse matrices. These are currently implemented using `scipy.sparse.csr_matrix`, and can give quite the speedup in the right circumstances. Let's look at the case of when there are `numEntries` nonzero elements in each Hamiltonian component, while also expanding the Hilbert space dimension substantially. 

In [13]:
numEntries = 64

new_state_vec = r(n**2)

den_ham_ops = []
sps_ham_ops = []
for i in range(m):
    i1 = np.random.choice(np.arange(n),size=(numEntries),replace=False)
    i2 = np.random.choice(np.arange(n),size=(numEntries),replace=False)
    d = r(numEntries)
    sph = csr_matrix((d,(i1,i2)),shape=(n**2,n**2))
    sps_ham_ops.append(sph)
    den_ham_ops.append(sph.toarray())
den_ham_ops = np.array(den_ham_ops)
sps_ham_ops = np.array(sps_ham_ops)

sigVals = r(1,m)

# Hamiltonian calculation
%timeit np.tensordot(sigVals,den_ham_ops,axes=1)
%timeit np.tensordot(sigVals,sps_ham_ops,axes=1)

154 ms ± 860 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
513 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


While redoing these calculations with either a smaller Hilbert space or more nonzero entries will sometimes yield results favoring dense calculation, in cases where it's appropriate, sparse calculation can yield great dividends. 

With that said, the way that sparse matrix calculations are done does lead to some very weird calculation methods being favored. In particular, consider the case of trying to evaluate some Hamiltonian as $Hv=(\sum_j s_j H_j)v$ where the $H_j$ are sparse. Broadly speaking, there are two approaches: either compute $H$ first, then compute $Hv$, or compute $H_jv$ for all $j$ first, then compute $\sum_j s_j(H_jv)$. It turns out that there is a significant speedup to be had: 

In [14]:
# Compute H, then Hv
%timeit tmp = np.tensordot(sigVals,sps_ham_ops,axes=1)[0]; tmp.dot(new_state_vec)
# Compute H_jv for all j first, then dot with signal values.
%timeit tmp = np.empty(shape=(1),dtype="O"); tmp[0]=new_state_vec; np.dot(sigVals,sps_ham_ops*tmp)[0]

# This is the odd way that is more efficient to compute these values
tmp = np.empty(shape=(1),dtype="O")
tmp[0] = new_state_vec
cf(np.dot(sigVals,sps_ham_ops*tmp)[0],np.tensordot(sigVals,sps_ham_ops,axes=1)[0].dot(new_state_vec))

628 µs ± 59.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
79.8 µs ± 695 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
True


What happens when we have a matrix of states? Intuition might suggest that the same trend should hold true, but it very much does not: 

In [15]:
new_state_vecs = r(n**2,n**2)

# Compute H, then Hv
%timeit tmp = np.tensordot(sigVals,sps_ham_ops,axes=1)[0]; tmp.dot(new_state_vecs)
# Compute H_jv for all j first, then dot with signal values.
%timeit tmp = np.empty(shape=(1),dtype="O"); tmp[0]=new_state_vecs; np.dot(sigVals,sps_ham_ops*tmp)[0]

# This is the odd way that is more efficient to compute these values
tmp = np.empty(shape=(1),dtype="O")
tmp[0] = new_state_vecs
cf(np.dot(sigVals,sps_ham_ops*tmp)[0],np.tensordot(sigVals,sps_ham_ops,axes=1)[0].dot(new_state_vecs))

42 ms ± 920 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
620 ms ± 5.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
True


In this case, it is _much_ faster to compute the Hamiltonian first, then take the dot product. Because the choice of implementation method is both significant in determining the speed of the computation and has an effectiveness that depends on whether we are evaluating matrices or single vectors, we have chosen to implement both systems, checking before doing the computation whether we're working with a vector or a matrix. 